In [ ]:
from typing import Any, List
from llama_index.core import SimpleDirectoryReader
from llama_index.core.ingestion import IngestionCache,IngestionPipeline
from llama_index.core.schema import BaseNode, TransformComponent
from llama_index.core.extractors import SummaryExtractor,QuestionsAnsweredExtractor

class MyNodeTransformer(TransformComponent):
    def __call__(self, nodes: List[BaseNode], **kwargs: Any) -> List[BaseNode]:
        # nodes = super().__call__(nodes, **kwargs)
        # nodes = [{**n, 'metadata':{**n.get('metadata',{}),'author':'sumudu'}} for n in nodes]
        return super().__call__(nodes, **kwargs)

cache_persist_path="../store/ingestionCache.json"

reader = SimpleDirectoryReader(input_dir="../docs/small")

documents = reader.load_data()

try:
    chached = IngestionCache.from_persist_path(persist_path=cache_persist_path)
    print("Cached file discovered.....")
except:
    chached=""
    print("No cached found.....")

pipeline = IngestionPipeline(
    transformations=[
        # MyNodeTransformer(),
        SummaryExtractor(),
        QuestionsAnsweredExtractor(questions=2)
    ],
    cache=chached
)

nodes = pipeline.run(
    show_progress=True,
    documents=documents
)
pipeline.cache.persist(cache_persist_path)
print("Pipeline completed...")

for d in nodes[0:10]:
    print(d.metadata)